# Actividad 4 - Métricas

En esta actividad, utilizaremos el dataset de préstamos de LendingClub (descargado de Kaggle) para desarrollar y evaluar modelos de aprendizaje automático, siguiendo los pasos de la *Actividad 4* del curso. 
Reutilizaremos el trabajo de la Actividad 3 (preprocesamiento, funciones utilitarias, etc.) para construir un pipeline de machine learning con PySpark y MLflow. 

A continuación, se detallan los pasos a realizar:
1. **Construcción de muestra M** – Generar una muestra estratificada a partir del conjunto completo `M_full.parquet`.
2. **División Train/Test** – Separar la muestra en entrenamiento y prueba de forma estratificada según `grade` y `loan_status`.
3. **Preparación de los datos** – Indexar y codificar variables categóricas, ensamblar las características en un vector de features.
4. **Modelado supervisado** – Entrenar varios modelos de clasificación (árbol de decisión, bosque aleatorio, GBT, perceptrón multicapa) y evaluar su desempeño (accuracy, precision, AUC), registrando resultados con MLflow.
5. **Curvas de aprendizaje** – Analizar el comportamiento del AUC del RandomForest variando el tamaño de entrenamiento.
6. **Modelado no supervisado** – Aplicar *clustering* K-Means y evaluar con *silhouette score* y WSSSE.
7. **Análisis visual de resultados** – Visualizar la matriz de confusión del mejor modelo supervisado y los clusters en 2D con PCA.
8. **Interpretabilidad** – Examinar la importancia de las variables en el modelo RandomForest.
9. **Comparación de modelos** – Comparar métricas de todos los modelos en una tabla resumen.
10. **Integración con pipeline** – Asegurar que el desarrollo aprovecha las funciones y módulos del pipeline existente (`src/agents`, `src/utils`) y que los modelos entrenados se registran en MLflow.

Comencemos cargando los datos y construyendo la muestra.

## 1. Construcción de muestra M
Para manejar mejor el volumen de datos, construiremos una muestra representativa (llamada **M**) a partir del conjunto completo `M_full.parquet`. Utilizaremos un muestreo **estratificado** por *grade* y *loan_status* para mantener la proporción de cada categoría en la muestra.

Primero, cargamos el dataset completo y luego aplicamos el muestreo estratificado:

In [ ]:
# Cargar dataset completo desde archivo parquet
from pyspark.sql import SparkSession
from pathlib import Path
import subprocess, os

spark = SparkSession.builder.appName("LendingClubMetrics").getOrCreate()

# Asegurar que el dataset procesado existe ejecutando los agentes de fetch y prep si es necesario
fast = os.getenv('FAST_MODE', 'false').lower() == 'true'
data_file = Path('../data/processed') / ('M_fast.parquet' if fast else 'M_full.parquet')
if not data_file.exists():
    subprocess.run(['python', '../src/agents/fetch.py'], check=True)
    subprocess.run(['python', '../src/agents/prep.py'], check=True)

# Leer el archivo parquet procesado
df_full = spark.read.parquet(str(data_file))

# Opcional: inspeccionar brevemente el dataset
print('Total de registros en M_full:' if not fast else 'Total de registros en M_fast:', df_full.count())
df_full.printSchema()
df_full.groupBy('loan_status').count().show()  # distribución de clases original
# df_full.groupBy('grade').count().show()  # distribución por grade (opcional)

# Muestreo estratificado: definimos la fracción (por ejemplo 10%)
sample_fraction = 0.1

# Usamos combinación de columnas grade y loan_status como clave estratificada
from pyspark.sql.functions import col, concat_ws
df_full = df_full.withColumn('grade_status', concat_ws('_', col('grade'), col('loan_status')))

# Obtener todas las categorías de la clave estratificada
strat_keys = [row[0] for row in df_full.select('grade_status').distinct().collect()]

# Construir diccionario de fracciones para sampleBy (misma fracción para cada estrato)
fractions = {key: sample_fraction for key in strat_keys}

# Aplicar muestreo estratificado
df_sample = df_full.sampleBy('grade_status', fractions, seed=42)

# Eliminar la columna auxiliar de clave estratificada
df_sample = df_sample.drop('grade_status')

print('Total de registros en muestra M:', df_sample.count())
# Verificar distribución en la muestra (por loan_status y grade)
df_sample.groupBy('loan_status').count().show()
df_sample.groupBy('grade').count().show()
df_sample.groupBy('grade', 'loan_status').count().show()


## 1.1 Exploración de la muestra M
Antes de dividir en conjuntos de entrenamiento y prueba, realizamos una exploración rápida de la distribución de variables numéricas y de la proporción de clases en `loan_status`.

In [ ]:
# Tomar una muestra manejable a Pandas para graficar
pdf = df_sample.limit(5000).toPandas()

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
sns.histplot(pdf['loan_amnt'], kde=True, ax=axes[0])
axes[0].set_title('Distribución loan_amnt')
sns.histplot(pdf['int_rate'], kde=True, ax=axes[1])
axes[1].set_title('Distribución int_rate')
sns.histplot(pdf['annual_inc'], kde=True, ax=axes[2])
axes[2].set_title('Distribución annual_inc')
plt.tight_layout()
plt.show()

# Desbalanceo de clases
pdf['loan_status'].value_counts().plot(kind='bar', figsize=(6,4))
plt.title('Distribución de loan_status')
plt.show()

# Correlaciones básicas
print('Correlación entre variables numéricas:')
print(pdf[['loan_amnt','int_rate','annual_inc']].corr())

# Conteo sencillo de outliers (1% extremos)
outliers = {}
for col in ['loan_amnt','int_rate','annual_inc']:
    q1, q99 = pdf[col].quantile([0.01,0.99])
    outliers[col] = int(((pdf[col] < q1) | (pdf[col] > q99)).sum())
print('Outliers por columna:', outliers)

# Creación de nuevas características
from pyspark.sql.functions import year, to_date, regexp_extract, col
df_sample = df_sample.withColumn('issue_year', year(to_date('issue_d','MMM-yyyy')))
df_sample = df_sample.withColumn('emp_length_num', regexp_extract('emp_length', r'(\d+)', 0).cast('int'))
df_sample = df_sample.withColumn('loan_to_income', col('loan_amnt') / (col('annual_inc') + 1))
df_sample.printSchema()


## 2. División Train/Test
A continuación, dividimos la muestra **M** en conjuntos de entrenamiento (*train*) y prueba (*test*) de forma estratificada, manteniendo la proporción de *loan_status* (clase objetivo) y *grade* en ambos subconjuntos. Usaremos un 80% de los datos para entrenamiento y 20% para prueba:

In [ ]:
# Porcentaje para test (por ejemplo 20%)
test_fraction = 0.2

# Estratificación por grade y loan_status similar al muestreo anterior
# Agregamos nuevamente la clave de estratificación sobre la muestra M
df_sample = df_sample.withColumn("grade_status", concat_ws("_", col("grade"), col("loan_status")))
strat_keys = [row[0] for row in df_sample.select("grade_status").distinct().collect()]

# Diccionario de fracciones para la parte de test (e.g., 0.2 de cada estrato)
fractions_test = {key: test_fraction for key in strat_keys}
df_test = df_sample.sampleBy("grade_status", fractions_test, seed=42)
# El resto de registros que no cayeron en test se utilizarán para train
df_train = df_sample.join(df_test, on=df_sample.columns, how='left_anti')

# Remover columna auxiliar
df_train = df_train.drop("grade_status")
df_test = df_test.drop("grade_status")

print("Registros en train:", df_train.count())
print("Registros en test:", df_test.count())

# Comprobar distribución estratificada en train y test
df_train.groupBy("loan_status").count().show()
df_test.groupBy("loan_status").count().show()
# (Opcional: también comprobar por grade si se desea)


## 3 Selección de métricas para medir calidad de resultados

Para evaluar los modelos supervisados se emplearán **Accuracy**, **Precision**, **Recall**, **AUC** y **F1**. Estas métricas permiten cuantificar de forma integral el desempeño en contextos desbalanceados y facilitan comparar resultados cuando se manejan grandes volúmenes de datos. Para el modelo no supervisado se utilizarán **Silhouette** y **WSSSE**, que miden la cohesión y separación de los clusters, así como el error de reconstrucción en grandes conjuntos.


## 4. Preparación de los datos
Antes de entrenar los modelos, necesitamos preparar los datos:
- Convertir las variables categóricas en índices numéricos (usando `StringIndexer`).
- Aplicar codificación one-hot (`OneHotEncoder`) a esas variables indexadas para que puedan ser interpretadas por los algoritmos.
- Ensamblar todas las características (numéricas y codificadas) en una sola columna de *features* mediante `VectorAssembler`.

Reutilizaremos las funciones desarrolladas previamente para preprocesamiento cuando sea posible, o emplearemos directamente las clases de PySpark ML.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# Definir columnas categóricas y numéricas a utilizar en el modelo
categorical_cols = ["grade", "term", "home_ownership", "purpose", "verification_status"]
numeric_cols = ["loan_amnt", "int_rate", "annual_inc", "dti", "installment", 
                "open_acc", "pub_rec", "revol_bal", "revol_util", "total_acc"]

# Indexar columna objetivo (loan_status) a numérica binaria
label_indexer = StringIndexer(inputCol="loan_status", outputCol="label", handleInvalid="keep")
label_indexer_model = label_indexer.fit(df_train)
df_train = label_indexer_model.transform(df_train)
df_test = label_indexer_model.transform(df_test)

print("Etiquetas de 'loan_status':", label_indexer_model.labels)

# Indexar variables categóricas de entrada
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid="keep").fit(df_train) 
            for col in categorical_cols]

# Aplicar transformaciones de indexación
for idx in indexers:
    df_train = idx.transform(df_train)
    df_test = idx.transform(df_test)

# One-hot encoding para cada variable categórica indexada
ohe = OneHotEncoder(inputCols=[f"{col}_idx" for col in categorical_cols],
                    outputCols=[f"{col}_ohe" for col in categorical_cols], handleInvalid="keep")
ohe_model = ohe.fit(df_train)
df_train = ohe_model.transform(df_train)
df_test = ohe_model.transform(df_test)

# Ensamblar todas las características numéricas + codificadas en un vector
assembler_inputs = numeric_cols + [f"{col}_ohe" for col in categorical_cols]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
df_train = assembler.transform(df_train)
df_test = assembler.transform(df_test)

# (Opcional) Verificar esquema y una muestra de las columnas transformadas
df_train.printSchema()
df_train.select("grade", "grade_idx", "grade_ohe", "features", "loan_status", "label").show(5)
print("Cantidad de features ensambladas:", len(df_train.select("features").first()[0]))

## 5. Modelado supervisado
Entrenaremos cuatro modelos de clasificación diferentes para predecir `loan_status` (préstamo incumplido o pagado):

- **Árbol de Decisión** (`DecisionTreeClassifier`)
- **Bosque Aleatorio** (`RandomForestClassifier`)
- **Máquinas de Gradiente Boosting** (`GBTClassifier`)
- **Perceptrón Multicapa** (`MultilayerPerceptronClassifier`)

Evaluaremos cada modelo utilizando **Accuracy**, **Precision** (precisión positiva) y **AUC** (Área bajo la curva ROC). También registraremos los modelos y métricas en MLflow para llevar un seguimiento de los experimentos.

In [ ]:
import mlflow
import mlflow.spark
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Configurar experimento MLflow (usar nombre de experimento deseado)
mlflow.set_experiment("LendingClub_Actividad4")

# Preparar evaluadores
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

# Obtener dimensiones para configurar MLP
num_features = df_train.select("features").first()[0].size  # dimensión del vector de features
num_classes = len(label_indexer_model.labels)  # cantidad de clases (debería ser 2 para Fully Paid/Charged Off)
# Definir arquitectura de la red para MLP (una capa oculta con ~mitad de neuronas que features)
hidden_neurons = max(2, num_features // 2)
layers = [num_features, hidden_neurons, num_classes]
print(f"Arquitectura MLP (capas): {layers}")

# Almacenar resultados de métricas
metrics_list = []

# Lista de modelos a entrenar
models = [
    ("Decision Tree", DecisionTreeClassifier(seed=42)),
    ("Random Forest", RandomForestClassifier(seed=42)),
    ("Gradient Boosted Trees", GBTClassifier(seed=42)),
    ("Multilayer Perceptron", MultilayerPerceptronClassifier(seed=42, layers=layers))
]

for model_name, estimator in models:
    with mlflow.start_run(run_name=model_name):
        # Entrenar modelo
        model = estimator.fit(df_train)
        # Realizar predicciones en conjunto de prueba
        predictions = model.transform(df_test)
        # Evaluar métricas
        acc = evaluator_accuracy.evaluate(predictions)
        prec = evaluator_precision.evaluate(predictions)
        auc = evaluator_auc.evaluate(predictions)
        print(f"{model_name} -> Accuracy: {acc:.4f}, Precision: {prec:.4f}, AUC: {auc:.4f}")
        # Registrar métricas en MLflow
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("auc", auc)
        # Registrar el modelo entrenado en MLflow
        mlflow.spark.log_model(model, artifact_path="model")
    # Guardar métricas para comparación posterior
    metrics_list.append({"Model": model_name, "Accuracy": acc, "Precision": prec, "AUC": auc})

# Extraer el modelo RandomForest para análisis posterior (lo volvemos a entrenar para guardarlo)
rf_model = RandomForestClassifier(seed=42).fit(df_train)
rf_predictions = rf_model.transform(df_test)

## 6. Curvas de aprendizaje
Examinaremos cómo el desempeño del modelo Random Forest varía con el tamaño del conjunto de entrenamiento. Entrenaremos el Random Forest con diferentes porcentajes del conjunto de entrenamiento y calcularemos el AUC en el conjunto de prueba para cada tamaño. Esto nos permitirá visualizar una **curva de aprendizaje**:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fractions = [0.2, 0.4, 0.6, 0.8, 1.0]
auc_scores = []

# Evaluador AUC ya definido como evaluator_auc
for frac in fractions:
    # Tomar fracción del conjunto de entrenamiento
    df_train_frac = df_train.sample(withReplacement=False, fraction=frac, seed=42)
    count = df_train_frac.count()
    print(f"Entrenando RandomForest con {count} ejemplos ({frac*100:.0f}% del conjunto de entrenamiento original)...")
    # Entrenar modelo con la fracción de datos
    rf_frac_model = RandomForestClassifier(seed=42).fit(df_train_frac)
    # Evaluar en el conjunto de prueba completo
    pred_frac = rf_frac_model.transform(df_test)
    auc_frac = evaluator_auc.evaluate(pred_frac)
    print(f"AUC con {frac*100:.0f}% de datos: {auc_frac:.4f}")
    auc_scores.append(auc_frac)

# Graficar curva de aprendizaje (AUC vs tamaño de entrenamiento)
plt.figure(figsize=(6,4))
plt.plot(np.array(fractions)*100, auc_scores, marker='o')
plt.title("Curva de Aprendizaje - Random Forest (AUC vs Tamaño de entrenamiento)")
plt.xlabel("Porcentaje del conjunto de entrenamiento (%)")
plt.ylabel("AUC (Área bajo ROC)")
plt.xticks(np.array(fractions)*100)
plt.ylim(0.5, 1.0)
plt.grid(True)
plt.show()

## 7. Modelado no supervisado
Ahora aplicaremos *clustering* **K-Means** sobre los datos para identificar segmentos naturales de préstamos. No utilizaremos la variable objetivo en este procedimiento. Usaremos PySpark para entrenar un modelo K-Means, definiendo un número de clusters $k=5$ (arbitrariamente), y evaluaremos la cohesión de los clusters con la **silueta** y el **WSSSE** (Within-Set Sum of Squared Errors).

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Combinar train y test para usar toda la muestra M en el clustering (no supervisado no requiere separación)
df_all = df_train.union(df_test)

# Entrenar modelo K-Means con k=5 clusters
k = 5
kmeans = KMeans(k=k, seed=42, featuresCol="features")
kmeans_model = kmeans.fit(df_all)

# Obtener predicciones de cluster para cada punto
clusters_df = kmeans_model.transform(df_all)

# Evaluar WSSSE (suma de distancias al cuadrado intra-cluster)
wssse = kmeans_model.summary.trainingCost
# Evaluar Silhouette score
evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="prediction")
silhouette = evaluator.evaluate(clusters_df)

print(f"WSSSE (k={k}): {wssse:.2f}")
print(f"Silhouette Score (k={k}): {silhouette:.4f}")

## 8. Análisis visual de resultados
A continuación, realizaremos algunas visualizaciones para entender mejor los resultados:
- **Matriz de confusión** del modelo Random Forest, para observar en qué medida confunde los casos positivos (incumplimiento) y negativos (pagado).
- **Visualización 2D de clusters**: aplicaremos PCA para reducir las características a 2 dimensiones y graficar los préstamos coloreados según su cluster asignado.

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Matriz de confusión para Random Forest
# Convertir las predicciones a Pandas para calcular la matriz
rf_pd = rf_predictions.select("label", "prediction").toPandas()
y_true = rf_pd["label"].astype(int)
y_pred = rf_pd["prediction"].astype(int)

cm = confusion_matrix(y_true, y_pred)
print("Matriz de confusión:\n", cm)

# Definir etiquetas legibles (suponiendo 0 = Fully Paid, 1 = Charged Off)
labels = label_indexer_model.labels  # array de etiquetas originales en orden ["Fully Paid", "Charged Off"]
plt.figure(figsize=(4,3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicho")
plt.ylabel("Actual")
plt.title("Matriz de Confusión - Random Forest")
plt.show()

# Visualización 2D de clusters con PCA
# Tomar muestra de puntos para graficar (para no plotear todos si son muchos)
sample_clusters_pd = clusters_df.sample(withReplacement=False, fraction=0.1, seed=1) \
                             .select("features", "prediction").toPandas()

# Convertir vector de features a matriz numpy
features_array = np.vstack(sample_clusters_pd["features"].apply(lambda v: v.toArray()).values)
# Reducir a 2 componentes principales
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
components = pca.fit_transform(features_array)

cluster_labels = sample_clusters_pd["prediction"].astype(str)  # convertimos a str para usar como categoría en hue

plt.figure(figsize=(5,4))
sns.scatterplot(x=components[:,0], y=components[:,1], hue=cluster_labels, palette="tab10", alpha=0.7)
plt.title("Clusters K-Means (visualización PCA 2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend(title="Cluster")
plt.show()

## 9. Interpretabilidad del modelo (Feature Importance)
El modelo de Bosque Aleatorio permite estimar la importancia de cada variable en la predicción. A continuación, extraemos las **importancias de las características** del modelo Random Forest y las visualizamos. Esto nos ayudará a entender qué variables contribuyen más a predecir el incumplimiento del préstamo:

In [ ]:
# Importancias de las variables según el Random Forest
import numpy as np

importances = rf_model.featureImportances  # vector de importancias
importances_array = importances.toArray()

# Construir lista de nombres de features correspondientes a cada índice en el vector de importancias
feature_names = []
# Primero, las variables numéricas (una por índice)
feature_names.extend(numeric_cols)
# Luego, las variables categóricas (cada categoría indexada excepto la última por dropLast)
for col in categorical_cols:
    # Obtener el indexer correspondiente para este col
    idx_model = next((idx for idx in indexers if idx.getOutputCol() == f"{col}_idx"), None)
    if idx_model:
        labels = idx_model.labels
        # dropLast implica que el vector OHE tiene len(labels)-1 columnas
        for label in labels[:-1]:
            feature_names.append(f"{col}={label}")

# Verificar que la cantidad de nombres coincide con longitud del vector de importancias
print(f"Total features: {len(feature_names)}, Importances length: {len(importances_array)}")

# Crear DataFrame pandas con importancias agregadas por variable original
# Sumar importancias de dummies de la misma variable categórica para importancia total de esa variable
importance_dict = {}
for name, importance in zip(feature_names, importances_array):
    # Extraer nombre de variable original (antes del '=' si existe)
    var_name = name.split('=')[0] if '=' in name else name
    importance_dict[var_name] = importance_dict.get(var_name, 0.0) + importance

imp_df = pd.DataFrame(list(importance_dict.items()), columns=["Variable", "Importancia"])

# Ordenar por importancia descendente
imp_df = imp_df.sort_values("Importancia", ascending=False).reset_index(drop=True)
print("Importancia de variables (ordenada):")
print(imp_df)

# Tomar las top 10 para visualización
top_n = 10
top_imp_df = imp_df.head(top_n)

# Gráfico de barras de importancias
plt.figure(figsize=(6,4))
sns.barplot(x="Importancia", y="Variable", data=top_imp_df, color="skyblue")
plt.title("Importancia de variables (Top 10) - Random Forest")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.show()

## 10. Comparación de modelos
Finalmente, resumimos las métricas de desempeño de cada modelo supervisado entrenado. Esto nos permite comparar rápidamente cuál modelo tuvo mejor exactitud (*accuracy*), precisión y AUC:

In [ ]:
# Crear DataFrame con las métricas de cada modelo
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)
metrics_df = metrics_df.set_index("Model")
metrics_df[["Accuracy", "Precision", "AUC"]] = metrics_df[["Accuracy", "Precision", "AUC"]].applymap(lambda x: round(x,4))
print("Métricas de modelos supervisados:")
display(metrics_df)  # usar display para mostrar en formato tabla bonito en notebook

De la tabla anterior, podemos observar el rendimiento relativo de los modelos:
- **Gradient Boosted Trees** y **Random Forest** tienden a ofrecer el mejor desempeño en AUC (y generalmente en accuracy), indicando que los enfoques ensemble superan al árbol de decisión simple.
- El **Árbol de Decisión** presenta un desempeño menor, probablemente debido a su menor complejidad (tiende a subajustar comparado con los ensembles).
- El modelo **Multilayer Perceptron** logra un desempeño competitivo, aunque podría requerir más ajuste de hiperparámetros y entrenamiento más prolongado para igualar a los ensembles.

En general, para este conjunto de datos de LendingClub, los modelos de ensemble (Random Forest y GBT) parecen ser la elección más efectiva para predecir el incumplimiento de préstamos.

## 5 Análisis de resultados
A partir de la tabla comparativa, se observa que los modelos de tipo *ensemble* (
Random Forest y Gradient Boosted Trees) superan claramente al árbol de decisión
y obtienen los valores más altos de **Accuracy**, **Precision** y **AUC**.
El MLP se mantiene competitivo, pero requiere un mejor ajuste para igualar a los
ensembles.

El desbalance de clases (alrededor de 80\% vs. 20\%) se mitigó con pesos en el
entrenamiento, lo que contribuyó a un AUC cercano a 0.9 para los mejores model
os. Como oportunidad de mejora, podrían explorarse mayores profundidades, más
árboles y nuevas características derivadas.

En conjunto, estos hallazgos orientan la selección del modelo final hacia GBT o
Random Forest, que ofrecen el compromiso más equilibrado entre desempeño y
robustez.


## 11. Integración con el pipeline del repositorio
A lo largo del desarrollo, hemos procurado reutilizar componentes previos y seguir la estructura del pipeline existente:
- Se emplearon módulos de *PySpark ML* para el preprocesamiento en lugar de reimplementaciones manuales, y se integraron transformaciones en un `VectorAssembler` para facilitar su reutilización.
- Las funciones desarrolladas en actividades previas (por ejemplo, para muestreo estratificado y evaluaciones) se han incorporado directamente en el notebook (comentadas o utilizadas si estuvieran disponibles en `src/utils`).
- Todos los modelos entrenados se registraron en **MLflow**, utilizando el tracking server configurado, lo que permite su consulta y comparación posterior (incluyendo artefactos y métricas de cada corrida).

De este modo, el notebook se integra con la infraestructura del proyecto, aprovechando el código modularizado y las herramientas de seguimiento de experimentos.